# Aplicación en imágenes de drones para agricultura de precisión
**Producto vectorial en 3D: normales, orientación foliar y áreas proyectadas**  
Este cuaderno muestra cómo:
1. Calcular **vectores normales** en una nube de puntos/mesh reconstruida desde drones (SfM/photogrametría) usando `Open3D`.
2. Estimar **orientación foliar** con respecto al **vector solar**.
3. Calcular **áreas proyectadas** en mapas topográficos (proyección al plano XY) y hacia el Sol.
Incluye una nube de puntos sintética `drone_sample_leaves.ply` para que puedas ejecutar sin datos externos.

## 0) Instalación (si hace falta)

In [ ]:
# Si estás en Colab, descomenta:
# !pip -q install open3d plotly numpy

## 1) Cargar datos (PLY)

In [ ]:
import numpy as np
import open3d as o3d
import plotly.graph_objects as go

# Cambia esta ruta si tienes tus propios datos (e.g., .ply exportado de Metashape, Meshroom, COLMAP, etc.)
ply_path = r"/mnt/data/drone_sample_leaves.ply"
pcd = o3d.io.read_point_cloud(ply_path)
print(pcd)

# Visual rápido en Open3D (opcional)
# o3d.visualization.draw_geometries([pcd])

## 2) Estimar normales con Open3D

In [ ]:
# Estima normales mediante vecinos más cercanos (KDTree)
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=30))

# Orienta las normales apuntándolas "hacia la cámara" situada por encima
pcd.orient_normals_towards_camera_location(camera_location=np.array([0,0,10.0]))

normals = np.asarray(pcd.normals)
points  = np.asarray(pcd.points)

print("Puntos:", points.shape, "Normales:", normals.shape)

## 3) Visualización 3D con Plotly (puntos + conos de normales)

In [ ]:
# Submuestrea para no saturar la escena con demasiados conos
idx = np.random.choice(points.shape[0], size=min(400, points.shape[0]), replace=False)
P = points[idx]
N = normals[idx]

# Escala de los conos
scale = 0.05

scatter = go.Scatter3d(
    x=points[:,0], y=points[:,1], z=points[:,2],
    mode="markers",
    marker=dict(size=2),
    name="Nube de puntos"
)

cones = go.Cone(
    x=P[:,0], y=P[:,1], z=P[:,2],
    u=N[:,0], v=N[:,1], w=N[:,2],
    sizemode="scaled",
    sizeref=scale,
    anchor="tail",
    showscale=False,
    name="Normales (a × b locales)"
)

fig = go.Figure(data=[scatter, cones])
fig.update_layout(
    title="Nube de puntos con normales (producto vectorial local)",
    scene=dict(aspectmode="data",
               xaxis_title="x", yaxis_title="y", zaxis_title="z")
)
fig.show()

## 4) Orientación foliar respecto al Sol (incidencia)

In [ ]:
# Define el vector solar a partir de azimut (° desde el Norte hacia el Este) y altura solar (° sobre el horizonte)
# Ajusta según tu ubicación y hora (puedes integrar librerías solares si lo deseas).
az_deg   = 135.0  # sureste
alt_deg  = 45.0   # 45° de altura
az = np.deg2rad(az_deg)
alt = np.deg2rad(alt_deg)

# Convención: x = Este, y = Norte, z = vertical (UP)
# Vector solar apuntando desde el punto hacia el Sol
s = np.array([
    np.sin(az)*np.cos(alt),  # Este
    np.cos(az)*np.cos(alt),  # Norte
    np.sin(alt)              # Up
], dtype=float)
s = s / np.linalg.norm(s)

# Coseno del ángulo de incidencia: dot(normal, s). Asumimos normales apuntando fuera de la hoja.
cos_inc = (normals @ s).clip(min=0)  # restringe a [0,1] para irradiancia efectiva
print("cos(incidencia):", cos_inc.min(), cos_inc.max())

# Visualiza puntos coloreados por cos(incidencia) (mayor es mejor alineación a la luz)
col = cos_inc
scatter_col = go.Scatter3d(
    x=points[:,0], y=points[:,1], z=points[:,2],
    mode="markers",
    marker=dict(size=2, color=col, colorscale="Viridis", showscale=True, colorbar=dict(title="cos θ")),
    name="Incidencia solar"
)

# Dibuja el vector solar en la escena
origin = points.mean(axis=0)
s_len = 0.5
solar_vec = go.Scatter3d(
    x=[origin[0], origin[0]+s_len*s[0]],
    y=[origin[1], origin[1]+s_len*s[1]],
    z=[origin[2], origin[2]+s_len*s[2]],
    mode="lines+markers+text",
    line=dict(width=8),
    marker=dict(size=3),
    text=["", "Sol"],
    name="Vector solar"
)

fig2 = go.Figure(data=[scatter_col, solar_vec])
fig2.update_layout(
    title=f"Orientación foliar vs Sol (az={az_deg}°, alt={alt_deg}°)",
    scene=dict(aspectmode="data",
               xaxis_title="x", yaxis_title="y", zaxis_title="z")
)
fig2.show()

## 5) Áreas proyectadas en mapas topográficos y hacia el Sol

In [ ]:
# Si no dispones de una malla, aproximamos áreas con triangulación de Delaunay en XY
from scipy.spatial import Delaunay

xy = points[:, :2]
tri = Delaunay(xy)
tris = tri.simplices  # índices de triángulos en 'points'

def triangle_area_and_normal(p0, p1, p2):
    v1 = p1 - p0
    v2 = p2 - p0
    n = np.cross(v1, v2)
    area = 0.5 * np.linalg.norm(n)
    n_hat = n / (np.linalg.norm(n) + 1e-12)
    return area, n_hat

A = 0.0          # área de la superficie (suma de triángulos 3D)
A_xy = 0.0       # área proyectada al plano XY (mapa topográfico)
A_sun = 0.0      # área proyectada ortogonal al Sol

k = np.array([0,0,1.0])    # normal del plano XY (mapa)
k = k / np.linalg.norm(k)

for t in tris:
    p0, p1, p2 = points[t[0]], points[t[1]], points[t[2]]
    area, n_hat = triangle_area_and_normal(p0, p1, p2)
    A += area
    A_xy += area * abs(np.dot(n_hat, k))     # proyección al plano XY
    A_sun += area * abs(np.dot(n_hat, s))    # proyección perpendicular a dirección solar

print(f"Área 3D (aprox): {A:.4f} m^2")
print(f"Área proyectada en XY (mapa): {A_xy:.4f} m^2")
print(f"Área proyectada hacia el Sol: {A_sun:.4f} m^2")

## 6) Exportar resultados (opcionales)

In [ ]:
# Guardar normales y cos(incidencia) como atributos CSV
import pandas as pd
df = pd.DataFrame({
    "x": points[:,0], "y": points[:,1], "z": points[:,2],
    "nx": normals[:,0], "ny": normals[:,1], "nz": normals[:,2]
})
df["cos_inc"] = (normals @ s).clip(min=0)
csv_path = "/mnt/data/normals_incidence.csv"
df.to_csv(csv_path, index=False)
csv_path